In [1]:
import os, random
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.fx.all import *

In [2]:
# specify input video file path
input_file = 'D:\MOVIES\Thor Love And Thunde 2022 [Hindi-English].mkv'

# specify output folder to save split videos
output_folder = './output/'

# specify duration of each split video (in seconds)
split_duration = 420

# specify noise intensity
noise_intensity = 0.05

In [ ]:
# create VideoFileClip object from input video
clip = VideoFileClip(input_file)

# calculate total duration of input video (in seconds)
total_duration = clip.duration

# calculate number of split videos required
num_splits = int(total_duration / split_duration) + 1

# loop through each split and save as separate mp4 file
for i in range(num_splits):
    # calculate start and end time for this split
    start_time = i * split_duration
    end_time = min((i + 1) * split_duration, total_duration)
    
    # extract video clip for this split
    subclip = clip.subclip(start_time, end_time)
    
    # specify output file name for this split
    output_file = os.path.join(output_folder, f'split_{i}.mp4')
    
    # write subclip to output file
    subclip.write_videofile(output_file, codec='libx264')
    
    # close subclip object to free up memory
    subclip.close()
    break

# close input clip object to free up memory
clip.close()

# print message when all splits are saved
print(f'{num_splits} split videos saved in {output_folder}')


In [3]:
import os
import random
import cv2
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.tools.cuts import find_video_period

# create VideoFileClip object from input video
clip = VideoFileClip(input_file)

# calculate total duration of input video (in seconds)
total_duration = clip.duration

# calculate number of split videos required
num_splits = int(total_duration / split_duration) + 1

# calculate period of video for the find_video_period function
period = find_video_period(clip)

# loop through each split and save as separate mp4 file
for i in range(num_splits):
    # calculate start and end time for this split
    start_time = i * split_duration
    end_time = min((i + 1) * split_duration, total_duration)
    
    # extract video clip for this split
    subclip = clip.subclip(start_time, end_time)
    
    # add horizontal flip effect to some frames
    flipped = [subclip.fl_image(lambda frame: cv2.flip(frame, 1)) if random.random() < 0.2 else subclip]
    
    # add random noise effect to all frames
    noisy = [cv2.addWeighted(frame, 1 - noise_intensity, 
                             noise_intensity * cv2.randn(frame.shape, 0, 255), 0, 0) for frame in flipped]

    # specify output file name for this split
    output_file = os.path.join(output_folder, f'split_{i}.mp4')
    
    # concatenate modified video frames and write to output file
    output_frames = [frame[:, :, ::-1] for frame in noisy]
    output_clip = ImageSequenceClip(output_frames, fps=clip.fps)
    output_clip.write_videofile(output_file, codec='libx264')
    
    # close subclip and output_clip objects to free up memory
    subclip.close()
    output_clip.close()
    break

# close input clip object to free up memory
clip.close()

# print message when all splits are saved
print(f'{num_splits} split videos saved in {output_folder}')
